In [10]:
#Script for compressing the geodatabase. 
#Includes logging tasks. 
#Also updates statistics on system tables. 
#January 11, 2024

import arcpy
import logging
from datetime import date

# Set the path to your geodatabase
geodatabase_path = "D:\\GDB_Connections\\cs_gdb1_sde.sde"
logFileLocation = r'D:\temp\compresslog_{}.txt'.format(date.today().strftime("_%d_%m_%Y"))

# Setup logging
logging.basicConfig(filename=logFileLocation + date.today().strftime("_%d_%m_%Y") + '.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(levelname)s %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S',
                        level=logging.INFO)

logging.info("Maintenance script starting")

try:
    # Check if the geodatabase exists
    if arcpy.Exists(geodatabase_path):
        #Sometimes a stale connection can cause issues with versioning.
        arcpy.ClearWorkspaceCache_management()

        # Block new connections to the database.
        logging.info("The database is no longer accepting connections")
        arcpy.AcceptConnections(geodatabase_path, False)

        # Disconnect all users from the database.
        logging.info("Disconnecting all users")
        arcpy.DisconnectUser(geodatabase_path, "ALL")
        
        # Compress the geodatabase
        logging.info("Running compress")
        arcpy.Compress_management(geodatabase_path)
        print("Geodatabase compressed successfully.")
        
        # Allow the database to begin accepting connections again
        logging.info("Allow users to connect to the database again")
        arcpy.AcceptConnections(geodatabase_path, True)
        
        #Analyze statistics on system tables
        logging.info("Updating statistics on the system tables")
        arcpy.AnalyzeDatasets_management(geodatabase_path, "SYSTEM", "", "ANALYZE_BASE", "ANALYZE_DELTA", "ANALYZE_ARCHIVE")
        print("Statistics updated successfully.")

    else:
        print("Geodatabase does not exist.")

except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))

except Exception as e:
    print(e)
finally: 
    logging.info("Script completed")
    

Geodatabase compressed successfully.
Statistics updated successfully.
